In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
API =  "EYFKAI04SV2M2AR4"

In [ ]:
# Task 1.

class ScriptData:

    def fetch_intraday_data(self,script):
        url= f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=5min&apikey={API}'
        r = requests.get(url)
        return r.json()
    
    def convert_intraday_data(self,script):
        data = self.fetch_intraday_data(script)["Time Series (5min)"]
        
        timestamp = list(data.keys())[::-1]
        
        value = list(data.values())[::-1]
        
        open_data = [float(value[i]['1. open']) for i in range(len(value))]
        high_data =  [float(value[i]['2. high']) for i in range(len(value))]
        low_data = [float(value[i]['3. low']) for i in range(len(value))]
        close_data = [float(value[i]['4. close']) for i in range(len(value))]
        volume_data =  [int(value[i]['5. volume']) for i in range(len(value))]
        
        df = pd.DataFrame(
            {
                "timestamp":timestamp,
                "open":open_data,
                "high":high_data,
                "low":low_data,
                "close":close_data,
                "volume":volume_data
            }
        )
        
        return df
        
    # Additional methods for overloading the following operations:
        # i. getitem
        # ii. setitem
        # iii. contains
    def getitem(self):
        pass
    
    def setitem(self):
        pass
    
    def contains(self):
        pass

In [ ]:
# Task 2.
        
def indicator1(df, timeperiod=5):
    time = list(df["timestamp"])
    closed = list(df["close"])
    # print(time)
    # print(closed)
    
    tim = []
    
    for i in range(1,len(closed)+1):
        if i<(timeperiod):
            tim.append("NaN")
        else:
            ad = float((sum(closed[i-5:i+1]))/timeperiod)
            tim.append(ad)
    
    indicator = pd.DataFrame(
        {
            "timestamp":time,
            "indicator":tim
        }
    )
    return indicator

In [ ]:
# Task 3.
class Strategy:
    def __init__(self,name):
        self.name = name
        
    def get_script_data(self):
        data = ScriptData()
        close_data = list(data.convert_intraday_data(self.name)['close'])[4:]
        indicator_data = list(indicator1(data.convert_intraday_data(self.name))['indicator'])[4:]
        #print(indicator_data,"\n",close_data)
        time = list(indicator1(data.convert_intraday_data(self.name))['timestamp'])[4:]
        return time, indicator_data, close_data
        
    def get_signals(self):
        
        (time,indicator_data,close_data) = self.get_script_data()
        
        graph_data = pd.DataFrame({
            "time":time,
            "indicator":indicator_data,
            "close":close_data
        })
        
        graph_data.plot()
        plt.show()
        
        signal_data = []
        
        
        for i in range(len(indicator_data)):
            if indicator_data[i]>close_data[i]:
                signal_data.append("BUY")
                
            elif indicator_data[i]<close_data[i]:
                signal_data.append("SELL")
                
            else:
                signal_data.append("NO_SIGNAL")
        
        signals = pd.DataFrame(
            {
                "timestamps":time,
                "signal":signal_data
            }
        )
        
        return signals
        

In [ ]:
script_data = ScriptData()

dataframe_1 = script_data.fetch_intraday_data("GOOGL")

dataframe_2 = script_data.convert_intraday_data("GOOGL")

In [ ]:
dataframe_3 = indicator1(script_data.convert_intraday_data("GOOGL"))


In [ ]:
print(dataframe_1)


In [ ]:
print(dataframe_2)

In [ ]:
print(dataframe_3)


In [ ]:
dataframe_2.plot()
plt.show()

In [ ]:
t = Strategy("GOOGL")
print(t.get_signals())